In [71]:
import pandas as pd
iris = pd.read_csv('https://raw.githubusercontent.com/edenLee94/analytics/main/iris_dataset/IRIS.csv')

In [72]:
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [73]:
iris['species'].value_counts()

Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: species, dtype: int64

In [74]:
iris['id'] = range(len(iris))

In [75]:
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species,id
0,5.1,3.5,1.4,0.2,Iris-setosa,0
1,4.9,3.0,1.4,0.2,Iris-setosa,1
2,4.7,3.2,1.3,0.2,Iris-setosa,2
3,4.6,3.1,1.5,0.2,Iris-setosa,3
4,5.0,3.6,1.4,0.2,Iris-setosa,4


In [76]:
# 순서를 바꿔주는게 더 좋을 것 같다.
iris = iris[['id', 'sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']]

In [77]:
iris.head()

,id,sepal_length,sepal_width,petal_length,petal_width,species
0,0,5.1,3.5,1.4,0.2,Iris-setosa
1,1,4.9,3.0,1.4,0.2,Iris-setosa
2,2,4.7,3.2,1.3,0.2,Iris-setosa
3,3,4.6,3.1,1.5,0.2,Iris-setosa
4,4,5.0,3.6,1.4,0.2,Iris-setosa


### KNN 학습을 위해서 train & test를 나눠준다!

In [78]:
iris_train = iris.sample(100,replace=False, random_state=2000).reset_index().drop(['index'],axis=1)

In [79]:
iris_train.head()

,id,sepal_length,sepal_width,petal_length,petal_width,species
0,19,5.1,3.8,1.5,0.3,Iris-setosa
1,84,5.4,3.0,4.5,1.5,Iris-versicolor
2,27,5.2,3.5,1.5,0.2,Iris-setosa
3,94,5.6,2.7,4.2,1.3,Iris-versicolor
4,128,6.4,2.8,5.6,2.1,Iris-virginica


참, 거짓값을 바꿔주는 것이다. 역접? 표현이 애매하지만, 참->거짓/거짓->참 이렇게 바꿔준다.
~iris['id'].isin(iris_train['id']

In [80]:
iris_test = iris.loc[~iris['id'].isin(iris_train['id']) ].reset_index().drop(['index'],axis=1)

In [81]:
# train: test = 2:1(100:50)
len(iris_test)

50

## KNN classfication하는 방법이다. 
왜냐하면, 종류를 구별하면 되는 문제이기 때문에 분류 문제 해결이다.

In [82]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier( n_neighbors=3)

In [83]:
knn.fit(iris_train[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']], iris_train['species'])

KNeighborsClassifier(n_neighbors=3)

In [84]:
predictions=knn.predict(iris_test[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']])

In [85]:
iris_test['pred'] = predictions

In [86]:
(iris_test['species'] == iris_test['pred']).mean()

0.94

### KNN의 최적의 값(k 값)을 찾아야한다.

In [87]:
for k in range(1,30):
    knn = KNeighborsClassifier( n_neighbors=k)
    knn.fit(iris_train[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']], iris_train['species'])
    predictions=knn.predict(iris_test[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']])
    iris_test['pred'] = predictions
    print((iris_test['species'] == iris_test['pred']).mean())

0.96
0.96
0.94
0.96
0.98
0.98
0.98
1.0
1.0
1.0
1.0
1.0
0.98
0.98
0.96
0.96
0.96
0.98
0.98
0.98
0.96
0.96
0.94
0.96
0.96
0.96
0.94
0.96
0.94


### <결과 해석>
k=8일때가 가장 최적의 k값이다. k가 증가할 수록, 모델을 학습하고, 예측하는 시간이 늘어나게 되어있다. -> 계산시간 단축하기 위해서

### <느낀점>
근데, 어떻게 1.0으로 정확한 값이 나올 수 있지? => 데이터 모집단 자체가 작아서 그럴 수 있긴하지만, 조금 찜찜하긴 하네

- train, test 데이터를 어떻게 나누는지에 따라서 결과가 달라질 수 있다. 이러면, 모델 성능 측정의 신뢰성이 있다고 할 수 있을까? -> 신뢰성 하락

# KNN Regression하는 방법이다.
왜냐하면, 연속되는 값을 예측하면 되는 문제이기 때문에 회귀 문제 해결이다.

In [88]:
del iris_train['species']
del iris_test['species']

In [89]:
iris_train.head()

,id,sepal_length,sepal_width,petal_length,petal_width
0,19,5.1,3.8,1.5,0.3
1,84,5.4,3.0,4.5,1.5
2,27,5.2,3.5,1.5,0.2
3,94,5.6,2.7,4.2,1.3
4,128,6.4,2.8,5.6,2.1


In [90]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=3)

In [91]:
knn.fit(iris_train[['sepal_length', 'sepal_width', 'petal_length']], iris_train['petal_width'])

KNeighborsRegressor(n_neighbors=3)

In [92]:
iris_test['pred'] = knn.predict(iris_test[['sepal_length', 'sepal_width', 'petal_length']])

In [93]:
iris_test

,id,sepal_length,sepal_width,petal_length,petal_width,pred
0,1,4.9,3.0,1.4,0.2,0.133333
1,2,4.7,3.2,1.3,0.2,0.233333
2,4,5.0,3.6,1.4,0.2,0.266667
3,8,4.4,2.9,1.4,0.2,0.200000
4,12,4.8,3.0,1.4,0.1,0.133333
5,14,5.8,4.0,1.2,0.2,0.233333
6,15,5.7,4.4,1.5,0.4,0.300000
7,16,5.4,3.9,1.3,0.4,0.200000
8,20,5.4,3.4,1.7,0.2,0.366667
9,21,5.1,3.7,1.5,0.4,0.233333


회귀 평가 기법인 'Mae' 기법을 사용한다.

In [94]:
abs(iris_test['petal_width']-iris_test['pred']).mean()

0.146

In [95]:
for k in range(1,30):
    knn = KNeighborsRegressor(n_neighbors=k)
    knn.fit(iris_train[['sepal_length', 'sepal_width', 'petal_length']], iris_train['petal_width'])
    iris_test['pred'] = knn.predict(iris_test[['sepal_length', 'sepal_width', 'petal_length']])
    print(abs(iris_test['petal_width']-iris_test['pred']).mean())

0.19400000000000003
0.172
0.146
0.1425
0.14640000000000003
0.1503333333333333
0.1477142857142857
0.14500000000000002
0.13755555555555557
0.13500000000000004
0.13454545454545458
0.13516666666666666
0.13953846153846153
0.14157142857142854
0.14266666666666666
0.14375
0.1448235294117647
0.14800000000000002
0.1487368421052632
0.1521
0.15257142857142858
0.15290909090909094
0.15226086956521745
0.15350000000000003
0.15488
0.15346153846153843
0.15444444444444447
0.15807142857142856
0.16172413793103446


값들 중에서 제일 작게 나온 결과값인 (0.13454545454545458) 이 값을 가지는 k가 최적의 해입니다!

## Model의 성능 측정하기

In [102]:
from sklearn.model_selection import cross_val_score
import numpy as np
for k in range(1,30):
    knn = KNeighborsClassifier(n_neighbors=k)
    print(np.mean(cross_val_score(knn, iris[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']], iris['species'],cv=3)) )

0.96
0.9466666666666667
0.9733333333333333
0.9866666666666667
0.98
0.9733333333333333
0.9733333333333333
0.9666666666666667
0.9733333333333333
0.9666666666666667
0.9666666666666667
0.9666666666666667
0.96
0.9533333333333333
0.9533333333333333
0.96
0.96
0.9533333333333333
0.9533333333333333
0.9466666666666667
0.9533333333333333
0.9466666666666667
0.94
0.9333333333333332
0.94
0.9533333333333333
0.9533333333333333
0.9533333333333333
0.9533333333333333


<strong>최적의 k가 4입니다! cv=n //n을 늘릴 수록, 신뢰성은 올라가지만, 엄청 느려진다.</strong>